In [ ]:
import gym

env = gym.make("CartPole-v0")
observation = env.reset()

for i in range(1000):
    env.render()
    action = env.action_space.sample() # your agent here (this takes random actions)
    observation, reward, done, info = env.step(action)

    if (i+1)%100 == 0:
        print(i+1, observation, reward, done, info)
        
    if done:
        observation = env.reset()

env.close()

100 [-0.11379394 -0.05179826  0.13815019  0.38340995] 1.0 False {}
200 [-0.0872247   0.78211613 -0.00569919 -1.22691841] 1.0 False {}
300 [-0.08352774 -1.16030341  0.21613116  1.99162   ] 1.0 True {}
400 [-0.20695815 -0.62769005  0.10958747  0.88750773] 1.0 False {}
500 [-0.14216951 -0.5649626  -0.05346357  0.00428762] 1.0 False {}
600 [-0.14212395 -0.407813    0.12835854  0.54606547] 1.0 False {}
700 [ 0.02105504  0.35522964 -0.0611438  -0.69441056] 1.0 False {}
800 [ 0.08317739 -0.39294913 -0.0397974   0.49970147] 1.0 False {}


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import gym

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import RMSprop

In [ ]:
model = Sequential()
model.add(Dense(16, activation='relu', input_shape=(4,)))
model.add(Dense(16, activation='relu'))
model.add(Dense(2, activation='linear')) # 0 or 1

model.compile(loss='mse', optimizer=RMSprop(lr=0.001)) # lr=0.001 (default)

In [ ]:
model.summary()

In [ ]:
s = np.array([1,0,1,0])

model.predict(s.reshape(1,4))

In [ ]:
ws = model.get_weights()

In [ ]:
ws[0]

In [ ]:
import collections

memory = collections.deque(maxlen = 10000)

In [ ]:
s = env.reset()

In [ ]:
for i in range(32):
    action = env.action_space.sample() # np.random.randint(2)
    s2, r, done, _ = env.step(action)
    
    memory.append([s, action, r, done, s2])
    
    s = env.reset() if done else s2 # done 이면 리셋

In [ ]:
len(memory)

In [ ]:
display(memory[0], memory[-1])

In [ ]:
epsilon = 1.0 # 처음에는 탐험만 수행한다. (랜덤 행동)
gamma = 0.99 # 감쇠율 (discount factor, 미래 보상을 얼마나 중요시할 지를 결정)
returns = [] # 에피소드 당 총보상값을 저장한다
count = 0

In [ ]:
### 총 10000번의 에피소드를 진행한다.
for episode in range(300):
    
    total_reward = 0 # 에피소스당 총보상값
    
    s = env.reset()
    
    for i in range(200): # 한 에피소드당 최대 200번만 행동한다
        
        ### 탐험 확률 지정 (1 부터 시작해서 점점 낮아지다 최소값은 1%)
        epsilon = 0.01 + (1-0.01)*np.exp(-0.0001*count)
        count += 1
        
        if np.random.rand() < epsilon:
            action = env.action_space.sample() # 랜덤 행동
        else:
            action = np.argmax(model.predict(s.reshape(1,4))[0]) # Q값이 높은 행동 선택
            
        s2, r, done, _ = env.step(action)
        
        total_reward += r
        
        memory.append([s, action, r, done, s2])
        
        ### 학습 (배치크기는 32)
        indices = np.random.choice(len(memory), 32, replace=False)
        samples = [memory[i] for i in indices]
        
        X = np.zeros([32,4]) # 입력 상태값
        y = np.zeros([32,2]) # 목표 Q값
        
        for i, sample in enumerate(samples): # sample -> [s,a,r,done,s2]
            X[i] = sample[0]
            y[i] = model.predict(sample[0].reshape(1,4))[0]
            
            if sample[3] == True: # done
                y[i][sample[1]] = sample[2]
            else:
                y[i][sample[1]] = sample[2] + gamma*np.max(model.predict(sample[-1].reshape(1,4))[0])
                
        model.fit(X, y, epochs=1, verbose=False)
        
        if done:
            break
        else:
            s = s2
            
    print('Episode: %d, Reward: %d, Epsilon: %.5f' % (episode+1, total_reward, epsilon))
    returns.append(total_reward)
    
    if total_reward == 200:
        break

In [ ]:
len(returns), len(memory)

In [ ]:
plt.plot(returns)

In [ ]:
for epoch in range(3):
    env.reset()
    
    done = False
    n = 0
    while not done:
        #a = np.random.randint(2)
        a = np.argmax(model.predict(s.reshape(1,4))[0])
        s, r, done, info = env.step(a)
        n += 1
        env.render()
        #print(s, r, done, info)
        
    print('Epoch: %d, Count: %d' % (epoch+1, n))
    print(s, r, done, info)
    #input()

In [ ]:
env.close()